In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
! pip install -U spacy -q

In [ ]:
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    3.3.1                         
Location         /usr/local/lib/python3.7/dist-packages/spacy
Platform         Linux-5.4.188+-x86_64-with-Ubuntu-18.04-bionic
Python version   3.7.13                        
Pipelines        en_core_web_sm (3.3.0)        



In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

In [ ]:
import json
f = open('/content/gdrive/MyDrive/bengali_food_training.json')
TRAIN_DATA = json.load(f)

In [ ]:
import json
f = open('/content/gdrive/MyDrive/bengali_food_validation.json')
VALIDATION_DATA = json.load(f)

In [ ]:
for text, annot in tqdm(TRAIN_DATA['annotations']): 
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc)

db.to_disk("./training_data.spacy") # save the docbin object

100%|██████████| 5/5 [00:00<00:00, 474.72it/s]


In [ ]:
for text, annot in tqdm(VALIDATION_DATA['annotations']): 
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc)

db.to_disk("./validation_data.spacy") # save the docbin object

100%|██████████| 5/5 [00:00<00:00, 1852.94it/s]


In [ ]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./validation_data.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-07-08 11:47:15,325] [INFO] Set up nlp object from config
[2022-07-08 11:47:15,338] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-07-08 11:47:15,343] [INFO] Created vocabulary
[2022-07-08 11:47:15,344] [INFO] Finished initializing nlp object
[2022-07-08 11:47:15,481] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     35.17    2.78    1.75    6.67    0.03
100     200          6.45    785.13  100.00  100.00  100.00    1.00
291     400          0.00      0.00  100.00  100.00  100.00    1.00
491     600          0.00      0.00  100.00  100

In [ ]:
nlp_ner = spacy.load("/content/model-best")

In [ ]:
doc = nlp_ner('''In Bangladesh, one can find various delicious curry, desserts and rice based meals.
Bangladeshi biryani is one of the most popular rice based meal. It is cooked with beef or chicken.
Hilsha fish is the national fish of Bangladesh. Hilsha curry is a very common dish. 
We eat Paratha with lentil or curry. Chingri malai curry is another popular dish made using
shrimp or prawn. We eat various cakes in special occasions.''')

In [ ]:
spacy.displacy.render(doc, style="ent", jupyter=True)